In [1]:
%cd ../../

/home/PJWSTK/s14028/engineer/Practical


In [2]:
import keras
import scipy.io as mat

from common import *
from augmentation import add_pmap
from augmentation import augmentation_data
from augmentation import augment_data

from CNN.CNN_v8 import CNN_v8

Using TensorFlow backend.


In [3]:
perspective = mat.loadmat("mall_dataset/perspective_roi.mat")["pMapN"]

perspective /= np.min(perspective)
perspective = np.round(perspective).astype(np.uint8)

train, test = data_sets()
image_tensors = train[0], test[0]
person_coo_tensors = train[1], test[1]
count_matrix = train[2], test[2]

image_train, image_test = image_tensors
person_coo_train, person_coo_test = person_coo_tensors
count_train, count_test = count_matrix
count_train = count_train.astype(np.uint16)
count_test = count_test.astype(np.uint16)

image_train = add_pmap(image_train, perspective)
image_test = add_pmap(image_test, perspective)

In [4]:
cnn_v8 = CNN_v8((480, 640, 4), split_into_parts=20)
cnn_v8.def_model()

In [5]:
images = cnn_v8._prepare_images(image_train)
anwsers = cnn_v8._prepare_anwsers(person_coo_train)

In [6]:
ones_count = (np.sum(anwsers == 1) * 0.25).astype(np.uint32)
zeros_count = (ones_count / 0.25 * 0.75).astype(np.uint32)
validation_length = (zeros_count + ones_count).astype(np.int32)

val_indices = np.concatenate([np.where(anwsers == 0)[0][:zeros_count],
                              np.where(anwsers == 1)[0][:ones_count]])

In [7]:
anwsers[val_indices[zeros_count:]] = 0

In [8]:
val_indices = -(images.shape[0] - val_indices)

In [9]:
generator = keras.preprocessing.image.ImageDataGenerator(rotation_range=20)

augmentation = augmentation_data(image_train, anwsers, 20)
augmented_data = augment_data(generator, augmentation, images, anwsers)
images, anwsers = augmented_data

In [10]:
anwsers[val_indices[zeros_count:]] = 1

images[-validation_length:], images[val_indices] = images[val_indices], images[-validation_length:]
anwsers[-validation_length:], anwsers[val_indices] = anwsers[val_indices], anwsers[-validation_length:]

In [11]:
parallel_model = keras.utils.multi_gpu_model(
        cnn_v8.model,
        gpus=2,
        cpu_merge=False)
parallel_model.compile(
        loss="binary_crossentropy", 
        optimizer="adam", 
        metrics=["accuracy"])

In [12]:
path = "CNN/CNN_v8/weights/cnn_v8_1e_6_{}"

class ModelSaver(keras.callbacks.Callback):
    __path: str
    __current_epoch: int
        
    def __init__(self, path: str, epoch: int = 0):
        self.__path = path
        self.__current_epoch = epoch
        
    def on_epoch_end(self, *args, **kwargs):
        epoch_path = self.__path.format(self.__current_epoch)
        self.__current_epoch += 1
        
        parallel_model.save_weights(epoch_path)

model_saver = ModelSaver(path, 5)

In [13]:
parallel_model.optimizer.lr.assign(1e-4)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=1e-04>

In [14]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=5000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 179s 144us/step - loss: 6.4010 - accuracy: 0.8888 - val_loss: 2.3206 - val_accuracy: 0.8097
Epoch 2/5
1244460/1244460 [==============================] - 176s 142us/step - loss: 1.2388 - accuracy: 0.9478 - val_loss: 0.8566 - val_accuracy: 0.8859
Epoch 3/5
1244460/1244460 [==============================] - 175s 141us/step - loss: 0.5837 - accuracy: 0.9587 - val_loss: 0.5637 - val_accuracy: 0.8959
Epoch 4/5
1244460/1244460 [==============================] - 175s 140us/step - loss: 0.3848 - accuracy: 0.9639 - val_loss: 0.4391 - val_accuracy: 0.8975
Epoch 5/5
1244460/1244460 [==============================] - 175s 140us/step - loss: 0.2859 - accuracy: 0.9663 - val_loss: 0.3671 - val_accuracy: 0.9042


In [15]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=5000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 175s 141us/step - loss: 0.2255 - accuracy: 0.9683 - val_loss: 0.3241 - val_accuracy: 0.9011
Epoch 2/5
1244460/1244460 [==============================] - 175s 140us/step - loss: 0.1869 - accuracy: 0.9697 - val_loss: 0.2819 - val_accuracy: 0.9121
Epoch 3/5
1244460/1244460 [==============================] - 175s 140us/step - loss: 0.1590 - accuracy: 0.9714 - val_loss: 0.2763 - val_accuracy: 0.9061
Epoch 4/5
1244460/1244460 [==============================] - 175s 140us/step - loss: 0.1395 - accuracy: 0.9728 - val_loss: 0.2465 - val_accuracy: 0.9167
Epoch 5/5
1244460/1244460 [==============================] - 175s 140us/step - loss: 0.1249 - accuracy: 0.9737 - val_loss: 0.2399 - val_accuracy: 0.9149


In [16]:
parallel_model.optimizer.lr.assign(1e-5)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=1e-05>

In [17]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=5000, epochs=10,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/10
1244460/1244460 [==============================] - 175s 140us/step - loss: 0.1100 - accuracy: 0.9776 - val_loss: 0.2149 - val_accuracy: 0.9249
Epoch 2/10
1244460/1244460 [==============================] - 175s 141us/step - loss: 0.1059 - accuracy: 0.9786 - val_loss: 0.2114 - val_accuracy: 0.9272
Epoch 3/10
1244460/1244460 [==============================] - 175s 140us/step - loss: 0.1035 - accuracy: 0.9793 - val_loss: 0.2094 - val_accuracy: 0.9269
Epoch 4/10
1244460/1244460 [==============================] - 175s 140us/step - loss: 0.1014 - accuracy: 0.9796 - val_loss: 0.2087 - val_accuracy: 0.9272
Epoch 5/10
1244460/1244460 [==============================] - 175s 140us/step - loss: 0.0999 - accuracy: 0.9797 - val_loss: 0.2018 - val_accuracy: 0.9302
Epoch 6/10
1244460/1244460 [==============================] - 175s 140us/step - loss: 0.0979 - accuracy: 0.9803 - val_loss: 0.1978 - val_accuracy: 0.9318
Epoch 7/10
1244460/12444

In [18]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=5000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 175s 140us/step - loss: 0.0901 - accuracy: 0.9812 - val_loss: 0.1849 - val_accuracy: 0.9352
Epoch 2/5
1244460/1244460 [==============================] - 175s 140us/step - loss: 0.0884 - accuracy: 0.9814 - val_loss: 0.1816 - val_accuracy: 0.9361
Epoch 3/5
1244460/1244460 [==============================] - 175s 141us/step - loss: 0.0870 - accuracy: 0.9816 - val_loss: 0.1804 - val_accuracy: 0.9361
Epoch 4/5
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0856 - accuracy: 0.9818 - val_loss: 0.1770 - val_accuracy: 0.9376
Epoch 5/5
1244460/1244460 [==============================] - 175s 140us/step - loss: 0.0842 - accuracy: 0.9821 - val_loss: 0.1724 - val_accuracy: 0.9399


In [19]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=5000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 175s 140us/step - loss: 0.0828 - accuracy: 0.9823 - val_loss: 0.1699 - val_accuracy: 0.9413
Epoch 2/5
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0818 - accuracy: 0.9825 - val_loss: 0.1670 - val_accuracy: 0.9416
Epoch 3/5
1244460/1244460 [==============================] - 175s 140us/step - loss: 0.0803 - accuracy: 0.9827 - val_loss: 0.1672 - val_accuracy: 0.9419
Epoch 4/5
1244460/1244460 [==============================] - 175s 140us/step - loss: 0.0792 - accuracy: 0.9829 - val_loss: 0.1635 - val_accuracy: 0.9425
Epoch 5/5
1244460/1244460 [==============================] - 175s 140us/step - loss: 0.0782 - accuracy: 0.9831 - val_loss: 0.1585 - val_accuracy: 0.9461


In [20]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=5000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 175s 140us/step - loss: 0.0770 - accuracy: 0.9833 - val_loss: 0.1588 - val_accuracy: 0.9461
Epoch 2/5
1244460/1244460 [==============================] - 175s 140us/step - loss: 0.0759 - accuracy: 0.9836 - val_loss: 0.1570 - val_accuracy: 0.9461
Epoch 3/5
1244460/1244460 [==============================] - 175s 141us/step - loss: 0.0750 - accuracy: 0.9837 - val_loss: 0.1555 - val_accuracy: 0.9464
Epoch 4/5
1244460/1244460 [==============================] - 175s 140us/step - loss: 0.0739 - accuracy: 0.9838 - val_loss: 0.1528 - val_accuracy: 0.9479
Epoch 5/5
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0729 - accuracy: 0.9842 - val_loss: 0.1504 - val_accuracy: 0.9489


In [21]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=5000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 175s 141us/step - loss: 0.0720 - accuracy: 0.9844 - val_loss: 0.1504 - val_accuracy: 0.9483
Epoch 2/5
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0711 - accuracy: 0.9845 - val_loss: 0.1459 - val_accuracy: 0.9508
Epoch 3/5
1244460/1244460 [==============================] - 175s 140us/step - loss: 0.0700 - accuracy: 0.9848 - val_loss: 0.1450 - val_accuracy: 0.9508
Epoch 4/5
1244460/1244460 [==============================] - 175s 140us/step - loss: 0.0692 - accuracy: 0.9850 - val_loss: 0.1395 - val_accuracy: 0.9540
Epoch 5/5
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0684 - accuracy: 0.9851 - val_loss: 0.1394 - val_accuracy: 0.9540


In [22]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=5000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0677 - accuracy: 0.9852 - val_loss: 0.1409 - val_accuracy: 0.9525
Epoch 2/5
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0669 - accuracy: 0.9854 - val_loss: 0.1356 - val_accuracy: 0.9552
Epoch 3/5
1244460/1244460 [==============================] - 175s 140us/step - loss: 0.0660 - accuracy: 0.9858 - val_loss: 0.1334 - val_accuracy: 0.9561
Epoch 4/5
1244460/1244460 [==============================] - 175s 140us/step - loss: 0.0652 - accuracy: 0.9858 - val_loss: 0.1356 - val_accuracy: 0.9549
Epoch 5/5
1244460/1244460 [==============================] - 175s 140us/step - loss: 0.0647 - accuracy: 0.9858 - val_loss: 0.1311 - val_accuracy: 0.9579


In [23]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=5000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 175s 140us/step - loss: 0.0637 - accuracy: 0.9862 - val_loss: 0.1290 - val_accuracy: 0.9579
Epoch 2/5
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0632 - accuracy: 0.9862 - val_loss: 0.1247 - val_accuracy: 0.9611
Epoch 3/5
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0624 - accuracy: 0.9864 - val_loss: 0.1299 - val_accuracy: 0.9572
Epoch 4/5
1244460/1244460 [==============================] - 175s 140us/step - loss: 0.0619 - accuracy: 0.9864 - val_loss: 0.1258 - val_accuracy: 0.9588
Epoch 5/5
1244460/1244460 [==============================] - 175s 140us/step - loss: 0.0612 - accuracy: 0.9867 - val_loss: 0.1206 - val_accuracy: 0.9619
